<a id='1'></a>
## 1 · Setup & Spark Initialisation

In [ ]:
# ── Imports ────────────────────────────────────────────────────────
import sys, os, warnings, re
warnings.filterwarnings('ignore')

PROJECT_ROOT = os.path.abspath('..')
sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
from IPython.display import display, HTML

# ── Style ──────────────────────────────────────────────────────────
sns.set_theme(style='whitegrid', palette='viridis', font_scale=1.1)
plt.rcParams.update({
    'figure.figsize': (14, 5),
    'figure.dpi': 110,
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
})

print('✅ Libraries loaded')

In [ ]:
# ── Spark Session (Delta-enabled) ─────────────────────────────────
from src.utils.spark_utils import get_spark_session

spark = get_spark_session(
    app_name='NB01-DataExploration',
    master='local[*]',
    driver_memory='4g',
)

print(f'✅ SparkSession ready  ·  v{spark.version}  ·  {spark.sparkContext.master}')

<a id='2'></a>
## 2 · Synthetic Data Generation

The framework's `data_generator` creates realistic Indian e-commerce data with **intentional quality issues**:

| Injection | Rate | Purpose |
|-----------|------|---------|
| Null values | ~5 % | Completeness testing |
| Negative order values | ~2 % | Anomaly detection |
| Extreme outliers (₹5L–₹20L) | ~0.5 % | Outlier detection |
| Duplicate customers | ~3 % | Identity resolution |
| PII in free-text | ~10–15 % | PII detection / masking |
| Invalid emails | ~2 % | Validity checks |
| Seasonal spikes | Festival periods | Temporal pattern analysis |

In [ ]:
from src.utils.data_generator import (
    generate_customers, generate_products, generate_orders,
    generate_reviews, generate_order_items,
)

N_CUST, N_PROD, N_ORD, N_REV, N_ITEM = 10_000, 2_000, 50_000, 20_000, 50_000

print('⏳ Generating synthetic datasets …')
customers_pdf = generate_customers(n=N_CUST)
products_pdf  = generate_products(n=N_PROD)
orders_pdf    = generate_orders(n=N_ORD,
                                customer_ids=customers_pdf['customer_id'].tolist(),
                                product_ids=products_pdf['product_id'].tolist())
reviews_pdf   = generate_reviews(n=N_REV,
                                 customer_ids=customers_pdf['customer_id'].tolist(),
                                 product_ids=products_pdf['product_id'].tolist())
items_pdf     = generate_order_items(n=N_ITEM,
                                     order_ids=orders_pdf['order_id'].tolist(),
                                     product_ids=products_pdf['product_id'].tolist())

datasets = {
    'Customers':   customers_pdf,
    'Products':    products_pdf,
    'Orders':      orders_pdf,
    'Reviews':     reviews_pdf,
    'Order Items': items_pdf,
}

total_rows = sum(len(d) for d in datasets.values())
print(f'\n{"═"*60}')
print(f'{"  DATASET SUMMARY":^60}')
print(f'{"═"*60}')
for name, df in datasets.items():
    print(f'  {name:15s}  {len(df):>10,} rows  ×  {len(df.columns):>3} cols')
print(f'{"─"*60}')
print(f'  {"TOTAL":15s}  {total_rows:>10,} rows')
print(f'{"═"*60}')

<a id='3'></a>
## 3 · Schema Inspection

Styled HTML tables showing column names, data types, null percentages, and sample values.

In [ ]:
def schema_card(name, pdf):
    """Render a styled HTML schema card."""
    rows_html = ''
    for col in pdf.columns:
        dtype = str(pdf[col].dtype)
        n_null = pdf[col].isna().sum()
        pct = n_null / len(pdf) * 100
        sample = str(pdf[col].dropna().iloc[0])[:45] if not pdf[col].dropna().empty else '—'
        clr = '#e74c3c' if pct > 5 else '#f39c12' if pct > 0 else '#27ae60'
        rows_html += f'''<tr>
          <td style="padding:5px 10px;border-bottom:1px solid #eee;">{col}</td>
          <td style="padding:5px 10px;border-bottom:1px solid #eee;"><code>{dtype}</code></td>
          <td style="padding:5px 10px;border-bottom:1px solid #eee;text-align:right;
              color:{clr};font-weight:bold;">{pct:.1f}%</td>
          <td style="padding:5px 10px;border-bottom:1px solid #eee;font-size:11px;
              color:#666;">{sample}</td></tr>'''
    display(HTML(f'''
    <div style="border:1px solid #ddd;border-radius:8px;margin:8px 0;overflow:hidden;">
      <div style="background:#2c3e50;color:white;padding:10px 14px;">
        <b>📋 {name}</b> — {len(pdf):,} rows × {len(pdf.columns)} columns</div>
      <table style="width:100%;border-collapse:collapse;">
        <tr style="background:#f8f9fa;font-weight:bold;">
          <td style="padding:6px 10px;">Column</td>
          <td style="padding:6px 10px;">Type</td>
          <td style="padding:6px 10px;text-align:right;">Null %</td>
          <td style="padding:6px 10px;">Sample</td></tr>
        {rows_html}
      </table></div>'''))

for name, pdf in datasets.items():
    schema_card(name, pdf)

<a id='4'></a>
## 4 · Null & Missing-Value Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, (name, pdf) in zip(axes, [('Customers', customers_pdf),
                                    ('Orders', orders_pdf),
                                    ('Reviews', reviews_pdf)]):
    null_pcts = (pdf.isna().sum() / len(pdf) * 100).sort_values(ascending=False)
    null_pcts = null_pcts[null_pcts > 0]
    if len(null_pcts) > 0:
        colors = ['#e74c3c' if v > 5 else '#f39c12' if v > 2 else '#3498db'
                  for v in null_pcts]
        null_pcts.plot.barh(ax=ax, color=colors, edgecolor='white')
        ax.set_xlabel('Null %')
    else:
        ax.text(0.5, 0.5, 'No nulls ✓', ha='center', va='center', fontsize=14,
                color='#27ae60', transform=ax.transAxes)
    ax.set_title(f'{name} — Null %')

plt.suptitle('Missing-Value Analysis (Red > 5 %, Orange > 2 %, Blue ≤ 2 %)',
             fontsize=13, fontweight='bold', y=1.03)
plt.tight_layout()
plt.show()

<a id='5'></a>
## 5 · Anomaly Detection — Negatives & Outliers

In [ ]:
negative  = orders_pdf[orders_pdf['order_value'] < 0]
extreme   = orders_pdf[orders_pdf['order_value'] > 500_000]
valid_val = orders_pdf[(orders_pdf['order_value'] >= 0) & (orders_pdf['order_value'] <= 500_000)]

print(f'📊 Order-Value Anomaly Report')
print(f'   Total orders:        {len(orders_pdf):>10,}')
print(f'   Negative values:     {len(negative):>10,}  ({len(negative)/len(orders_pdf)*100:.2f}%)')
print(f'   Extreme outliers:    {len(extreme):>10,}  ({len(extreme)/len(orders_pdf)*100:.2f}%)')
print(f'   Valid range:         {len(valid_val):>10,}  ({len(valid_val)/len(orders_pdf)*100:.2f}%)')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

ax1.hist(valid_val['order_value'].clip(upper=100_000), bins=80,
         color='#3498db', edgecolor='white', alpha=0.85)
ax1.set_xlabel('Order Value (₹)')
ax1.set_ylabel('Frequency')
ax1.set_title('Order Value Distribution (clipped ₹1L)')
ax1.xaxis.set_major_formatter(mtick.StrMethodFormatter('₹{x:,.0f}'))

bp = ax2.boxplot(orders_pdf['order_value'].values, vert=True, widths=0.5,
                 flierprops=dict(marker='.', markerfacecolor='red', markersize=2, alpha=0.3),
                 patch_artist=True)
bp['boxes'][0].set_facecolor('#3498db')
ax2.set_ylabel('Order Value (₹)')
ax2.set_title('Box Plot — Outlier Spread')
ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('₹{x:,.0f}'))

plt.tight_layout()
plt.show()

<a id='6'></a>
## 6 · PII Leakage in Free-Text Fields

The generator injects PII (email, phone, Aadhaar, PAN) into `delivery_instructions`, `review_text`, and `description` to test the detection pipeline.

In [ ]:
pii_patterns = {
    'Email':   r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}',
    'Phone':   r'(?:\+91[\-\s]?)?\d{10}',
    'Aadhaar': r'\d{4}\s\d{4}\s\d{4}',
    'PAN':     r'[A-Z]{5}\d{4}[A-Z]',
}

def scan_pii(series, patterns):
    texts = series.dropna().astype(str)
    return {k: int(texts.str.contains(v, regex=True, na=False).sum())
            for k, v in patterns.items()}

pii_results = pd.DataFrame({
    'Orders – delivery_instructions': scan_pii(orders_pdf['delivery_instructions'], pii_patterns),
    'Reviews – review_text':          scan_pii(reviews_pdf['review_text'], pii_patterns),
    'Products – description':         scan_pii(products_pdf['description'], pii_patterns),
})

fig, ax = plt.subplots(figsize=(12, 5))
pii_results.plot.bar(ax=ax, color=['#e74c3c', '#f39c12', '#9b59b6'], edgecolor='white')
ax.set_title('🔐 PII Leakage Detected in Free-Text Fields')
ax.set_ylabel('Records Containing PII')
ax.set_xlabel('PII Type')
ax.legend(title='Field', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

display(pii_results.style.set_caption('PII Leakage Summary').format('{:,}'))

<a id='7'></a>
## 7 · Temporal & Seasonal Patterns

Festival spikes: Diwali (4×), Navratri (3.5×), Independence Day (2×), Holi (2×)

In [ ]:
orders_pdf['order_date'] = pd.to_datetime(orders_pdf['order_timestamp']).dt.date
daily = orders_pdf.groupby('order_date').agg(
    count=('order_id', 'count'),
    avg_value=('order_value', 'mean'),
    revenue=('order_value', 'sum'),
).reset_index()
daily['order_date'] = pd.to_datetime(daily['order_date'])

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

ax1.fill_between(daily['order_date'], daily['count'], alpha=0.35, color='#3498db')
ax1.plot(daily['order_date'], daily['count'], color='#2980b9', lw=0.9)
ax1.set_ylabel('Orders / Day')
ax1.set_title('📈 Daily Order Volume — Festival Spikes Visible')

ax2.fill_between(daily['order_date'], daily['avg_value'], alpha=0.25, color='#e74c3c')
ax2.plot(daily['order_date'], daily['avg_value'], color='#c0392b', lw=0.9)
ax2.set_ylabel('Avg Order Value (₹)')
ax2.set_xlabel('Date')
ax2.set_title('💰 Average Order Value Over Time')
ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('₹{x:,.0f}'))

plt.tight_layout()
plt.show()

In [ ]:
# Day-of-week analysis
orders_pdf['dow'] = pd.to_datetime(orders_pdf['order_timestamp']).dt.day_name()
dow_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
dow_counts = orders_pdf['dow'].value_counts().reindex(dow_order)

fig, ax = plt.subplots(figsize=(10, 4))
colors_dow = ['#3498db']*5 + ['#e74c3c']*2
dow_counts.plot.bar(ax=ax, color=colors_dow, edgecolor='white')
ax.set_title('📅 Orders by Day of Week (red = weekend)')
ax.set_ylabel('Order Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

<a id='8'></a>
## 8 · Order Status & Payment Methods

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

status = orders_pdf['order_status'].value_counts()
ax1.pie(status, labels=status.index, autopct='%1.1f%%',
        colors=sns.color_palette('RdYlGn_r', len(status)),
        startangle=140, pctdistance=0.82,
        wedgeprops=dict(edgecolor='white', linewidth=1.5))
ax1.set_title('Order Status Distribution')

pay = orders_pdf['payment_method'].value_counts()
ax2.pie(pay, labels=pay.index, autopct='%1.1f%%',
        colors=sns.color_palette('Set2', len(pay)),
        startangle=140, pctdistance=0.82,
        wedgeprops=dict(edgecolor='white', linewidth=1.5))
ax2.set_title('Payment Method Distribution')

plt.tight_layout()
plt.show()

<a id='9'></a>
## 9 · Geographic Distribution

In [ ]:
city_dist = customers_pdf.groupby(['city', 'city_tier']).size().reset_index(name='count')
city_dist = city_dist.sort_values('count', ascending=True).tail(20)

fig, ax = plt.subplots(figsize=(12, 7))
tier_colors = city_dist['city_tier'].map({1:'#e74c3c', 2:'#f39c12', 3:'#3498db'})
ax.barh(city_dist['city'], city_dist['count'], color=tier_colors, edgecolor='white')
ax.set_xlabel('Number of Customers')
ax.set_title('🏙️ Customer Distribution by City (Top 20)')

from matplotlib.patches import Patch
ax.legend(handles=[
    Patch(fc='#e74c3c', label='Tier 1 (Metro)'),
    Patch(fc='#f39c12', label='Tier 2'),
    Patch(fc='#3498db', label='Tier 3'),
], title='City Tier', loc='lower right')
plt.tight_layout()
plt.show()

<a id='10'></a>
## 10 · Customer Segments & DPDP Consent

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

seg = customers_pdf['customer_segment'].value_counts()
ax1.pie(seg, labels=seg.index, autopct='%1.1f%%',
        colors=['#2ecc71','#3498db','#f39c12','#e74c3c'],
        startangle=140, wedgeprops=dict(edgecolor='white', linewidth=1.5))
ax1.set_title('Customer Segments')

# DPDP Act consent analysis
mkt_yes = customers_pdf['consent_marketing'].fillna(False).astype(bool).sum()
dp_yes  = customers_pdf['consent_data_processing'].fillna(True).astype(bool).sum()
consent_df = pd.DataFrame({
    'Marketing': [mkt_yes, len(customers_pdf)-mkt_yes],
    'Data Processing': [dp_yes, len(customers_pdf)-dp_yes],
}, index=['Consented','Declined'])
consent_df.plot.bar(ax=ax2, color=['#2ecc71','#3498db'], edgecolor='white')
ax2.set_title('🇮🇳 DPDP Act — Consent Status')
ax2.set_ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

<a id='11'></a>
## 11 · Product Categories & Reviews

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Top categories
cat = orders_pdf['product_category'].value_counts().head(15)
cat.plot.barh(ax=ax1, color=sns.color_palette('viridis', len(cat)), edgecolor='white')
ax1.set_xlabel('Number of Orders')
ax1.set_title('📦 Top 15 Product Categories')
ax1.invert_yaxis()

# Review ratings
rating = reviews_pdf['rating'].value_counts().sort_index()
rating.plot.bar(ax=ax2, color=['#e74c3c','#f39c12','#f1c40f','#2ecc71','#27ae60'],
               edgecolor='white')
ax2.set_xlabel('Star Rating')
ax2.set_ylabel('Count')
ax2.set_title('⭐ Review Rating Distribution')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

<a id='12'></a>
## 12 · Duplicate Customer Analysis

~3 % near-duplicate customers injected for identity resolution testing.

In [ ]:
email_dupes = customers_pdf[customers_pdf['email'].notna()]
email_dupe_n = email_dupes['email'].duplicated(keep=False).sum()

phone_dupes = customers_pdf[customers_pdf['phone'].notna()]
phone_dupe_n = phone_dupes['phone'].duplicated(keep=False).sum()

invalid_email_n = customers_pdf[
    customers_pdf['email'].notna() &
    ~customers_pdf['email'].str.contains(r'^[^@]+@[^@]+\.[^@]+$', regex=True, na=False)
].shape[0]

issues = pd.DataFrame([
    ('Email Duplicates',  email_dupe_n),
    ('Phone Duplicates',  phone_dupe_n),
    ('Invalid Emails',    invalid_email_n),
    ('Null Emails',       int(customers_pdf['email'].isna().sum())),
    ('Null Phones',       int(customers_pdf['phone'].isna().sum())),
    ('Null Aadhaar',      int(customers_pdf['aadhaar'].isna().sum())),
], columns=['Issue', 'Count'])

fig, ax = plt.subplots(figsize=(11, 4))
clrs = ['#e74c3c','#e74c3c','#f39c12','#3498db','#3498db','#3498db']
ax.barh(issues['Issue'], issues['Count'], color=clrs, edgecolor='white')
for i, v in enumerate(issues['Count']):
    ax.text(v + max(issues['Count'])*0.01, i, f'{v:,}', va='center', fontweight='bold', fontsize=10)
ax.set_xlabel('Records')
ax.set_title('🔍 Data Quality Issues — Customer Dataset')
plt.tight_layout()
plt.show()

<a id='13'></a>
## 13 · DataProfiler — Automated Profiling

The framework's `DataProfiler` class generates statistical profiles, detects PII columns, and produces visualisation charts.

In [ ]:
from src.quality.data_profiler import DataProfiler

profiler = DataProfiler(spark)

# Convert orders to Spark DataFrame for profiling
orders_sdf = spark.createDataFrame(orders_pdf.drop(columns=['order_date', 'dow'], errors='ignore'))

print('⏳ Running DataProfiler on orders …')
profile = profiler.profile_dataframe(orders_sdf, table_name='orders')

print(f'\n✅ Profile Complete')
print(f'   Table:        {profile["table_name"]}')
print(f'   Row count:    {profile["row_count"]:,}')
print(f'   Column count: {profile["column_count"]}')
print(f'   Timestamp:    {profile["timestamp"]}')

# Show per-column stats
col_stats = []
for col_name, info in profile['columns'].items():
    col_stats.append({
        'Column': col_name,
        'Type': info.get('data_type', ''),
        'Null %': f"{info.get('null_percentage', 0):.1f}%",
        'Distinct': info.get('distinct_count', ''),
        'PII Detected': ', '.join([k for k, v in info.get('pii_detected', {}).items() if v > 0]) or '—',
    })
display(pd.DataFrame(col_stats).style.set_caption('DataProfiler — Column Statistics'))

<a id='14'></a>
## 14 · Executive Summary Dashboard

In [ ]:
display(HTML(f'''
<div style="background:linear-gradient(135deg,#1a1a2e,#16213e,#0f3460);color:white;
            padding:30px;border-radius:12px;margin:10px 0;font-family:sans-serif;">
  <h2 style="margin:0 0 20px;text-align:center;">📊 Data Exploration — Executive Summary</h2>
  <div style="display:grid;grid-template-columns:repeat(3,1fr);gap:15px;">
    <div style="background:rgba(255,255,255,0.08);padding:18px;border-radius:10px;text-align:center;">
      <div style="font-size:11px;opacity:0.7;">TOTAL RECORDS</div>
      <div style="font-size:28px;font-weight:bold;">{total_rows:,}</div></div>
    <div style="background:rgba(255,255,255,0.08);padding:18px;border-radius:10px;text-align:center;">
      <div style="font-size:11px;opacity:0.7;">DATASETS</div>
      <div style="font-size:28px;font-weight:bold;">5</div></div>
    <div style="background:rgba(255,255,255,0.08);padding:18px;border-radius:10px;text-align:center;">
      <div style="font-size:11px;opacity:0.7;">AVG ORDER VALUE</div>
      <div style="font-size:28px;font-weight:bold;">₹{orders_pdf["order_value"].mean():,.0f}</div></div>
    <div style="background:rgba(231,76,60,0.2);padding:18px;border-radius:10px;text-align:center;">
      <div style="font-size:11px;opacity:0.7;">NEGATIVE VALUES</div>
      <div style="font-size:28px;font-weight:bold;color:#e74c3c;">{len(negative):,}</div></div>
    <div style="background:rgba(243,156,18,0.2);padding:18px;border-radius:10px;text-align:center;">
      <div style="font-size:11px;opacity:0.7;">EXTREME OUTLIERS</div>
      <div style="font-size:28px;font-weight:bold;color:#f39c12;">{len(extreme):,}</div></div>
    <div style="background:rgba(231,76,60,0.2);padding:18px;border-radius:10px;text-align:center;">
      <div style="font-size:11px;opacity:0.7;">PII LEAKAGE (ORDERS)</div>
      <div style="font-size:28px;font-weight:bold;color:#e74c3c;">
        {sum(scan_pii(orders_pdf["delivery_instructions"], pii_patterns).values()):,}</div></div>
    <div style="background:rgba(255,255,255,0.08);padding:18px;border-radius:10px;text-align:center;">
      <div style="font-size:11px;opacity:0.7;">CITIES COVERED</div>
      <div style="font-size:28px;font-weight:bold;">{customers_pdf["city"].nunique()}</div></div>
    <div style="background:rgba(255,255,255,0.08);padding:18px;border-radius:10px;text-align:center;">
      <div style="font-size:11px;opacity:0.7;">PRODUCT CATEGORIES</div>
      <div style="font-size:28px;font-weight:bold;">{orders_pdf["product_category"].nunique()}</div></div>
    <div style="background:rgba(255,255,255,0.08);padding:18px;border-radius:10px;text-align:center;">
      <div style="font-size:11px;opacity:0.7;">EMAIL DUPLICATES</div>
      <div style="font-size:28px;font-weight:bold;">{email_dupe_n:,}</div></div>
  </div>
  <p style="margin:20px 0 0;text-align:center;opacity:0.6;font-size:12px;">
    Proceed to <b>Notebook 02</b> for Data Quality & Great Expectations analysis</p>
</div>
'''))

In [ ]:
spark.stop()
print('✅ SparkSession stopped — Notebook 01 complete')